I have three folder BACTERIA, NORMAL, VIRUS
in this address
/content/drive/MyDrive/CS415Project/chest_xray/processed_train
Each od them contain 700 images
add new folder beside them with name train_all
and copy all image in this folders to that with ordering number 1 to 2100
I want to add noise to each images based on below condition
1- for each image select on type noise(gaussian, impulse, salt and pepper, speckle) and
add noise to each of the image and save as in folder train_all_noise with same name and order
in this root address
/content/drive/MyDrive/CS415Project/chest_xray/processed_train_noisy
prepare google colab code for me


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

In [4]:
import os
import cv2
import numpy as np
from random import choice
from tqdm import tqdm

# Define paths
root_path = "/content/drive/MyDrive/CS415Project/chest_xray"
processed_train_path = os.path.join(root_path, "processed_train")
processed_train_noisy_path = os.path.join(root_path, "processed_train_noisy")
train_all_path = os.path.join(processed_train_path, "train_all")
train_all_noise_path = os.path.join(processed_train_noisy_path, "train_all_noise")

# Create new folders if not already exist
os.makedirs(train_all_path, exist_ok=True)
os.makedirs(train_all_noise_path, exist_ok=True)

# Combine all images into train_all with sequential numbering
folders = ["BACTERIA", "NORMAL", "VIRUS"]
current_index = 1
image_paths = []

for folder in folders:
    folder_path = os.path.join(processed_train_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            src_path = os.path.join(folder_path, filename)
            dst_path = os.path.join(train_all_path, f"{current_index:04d}.jpg")
            image_paths.append((src_path, dst_path))
            current_index += 1

# Copy images to train_all folder
for src, dst in tqdm(image_paths, desc="Copying images"):
    img = cv2.imread(src)
    cv2.imwrite(dst, img)

# Define noise addition functions
def add_gaussian_noise(image):
    mean = 0
    std_dev = 25
    noise = np.random.normal(mean, std_dev, image.shape).astype(np.float32)
    noisy_image = cv2.add(image.astype(np.float32), noise)
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def add_salt_and_pepper_noise(image, amount=0.02):
    noisy_image = np.copy(image)
    num_salt = np.ceil(amount * image.size * 0.5).astype(int)
    num_pepper = np.ceil(amount * image.size * 0.5).astype(int)

    # Salt noise
    coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 255

    # Pepper noise
    coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = 0

    return noisy_image

def add_impulse_noise(image):
    noisy_image = np.copy(image)
    mask = np.random.choice([0, 1, 2], size=image.shape[:2], p=[0.98, 0.01, 0.01])
    noisy_image[mask == 1] = 255  # White noise
    noisy_image[mask == 2] = 0    # Black noise
    return noisy_image

def add_speckle_noise(image):
    noise = np.random.randn(*image.shape).astype(np.float32)
    noisy_image = image + image * noise * 0.5
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

# Add noise to images and save in train_all_noise
noise_types = [add_gaussian_noise, add_salt_and_pepper_noise, add_impulse_noise, add_speckle_noise]

for _, dst in tqdm(image_paths, desc="Adding noise"):
    image = cv2.imread(dst)
    noise_function = choice(noise_types)
    noisy_image = noise_function(image)
    noisy_dst = os.path.join(train_all_noise_path, os.path.basename(dst))
    cv2.imwrite(noisy_dst, noisy_image)

print("Processing complete!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CS415Project/chest_xray/processed_train/BACTERIA'

Summary of Changes:
Clean Images Directory: Now uses /content/drive/MyDrive/CS415Project/chest_xray/processed_train/train_all.
Noisy Images Directory: Uses /content/drive/MyDrive/CS415Project/chest_xray/processed_train_noisy/train_all_noise.
Output Directory: Saves all results to /content/drive/MyDrive/CS415Project/chest_xray/New_result_UNET_train.
Image Sorting: Ensures images are sorted by number for correspondence between clean and noisy images.
This code trains the U-Net model and visualizes results while saving metrics and outputs in the specified folder.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import pandas as pd
import matplotlib.pyplot as plt

# Parameters
BATCH_SIZE = 16
EPOCHS = 20
IMAGE_SIZE = (128, 128)  # Resize images to 128x128
BASE_DIR = "/content/drive/MyDrive/CS415Project/chest_xray"
CLEAN_IMAGE_DIR = os.path.join(BASE_DIR, "processed_train/train_all")
NOISY_IMAGE_DIR = os.path.join(BASE_DIR, "processed_train_noisy/train_all_noise")
OUTPUT_DIR = os.path.join(BASE_DIR, "New_result_UNET_train")

# Create output directory
def create_output_directory(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

# Load and preprocess images
def load_and_resize_images(image_dir, target_size=(128, 128), color_mode="grayscale"):
    image_list = []
    if not os.path.exists(image_dir):
        print(f"Error: Directory {image_dir} does not exist!")
        return np.array([])

    for filename in sorted(os.listdir(image_dir), key=lambda x: int(x.split('.')[0])):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = load_img(os.path.join(image_dir, filename), target_size=target_size, color_mode=color_mode)
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1] range
            if img_array.ndim == 3 and img_array.shape[-1] == 3:
                img_array = np.mean(img_array, axis=-1, keepdims=True)  # Convert RGB to grayscale
            image_list.append(img_array)

    images = np.array(image_list)
    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)
    return images

# Define U-Net model
def unet_model(input_shape=(128, 128, 1)):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Decoder
    u1 = layers.UpSampling2D((2, 2))(p4)
    u1 = layers.concatenate([u1, c4])
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

    u2 = layers.UpSampling2D((2, 2))(c5)
    u2 = layers.concatenate([u2, c3])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

    u3 = layers.UpSampling2D((2, 2))(c6)
    u3 = layers.concatenate([u3, c2])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

    u4 = layers.UpSampling2D((2, 2))(c7)
    u4 = layers.concatenate([u4, c1])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c8)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Train U-Net model
def train_model():
    # Load clean and noisy images
    clean_images = load_and_resize_images(CLEAN_IMAGE_DIR, IMAGE_SIZE)
    noisy_images = load_and_resize_images(NOISY_IMAGE_DIR, IMAGE_SIZE)

    assert clean_images.shape[0] == noisy_images.shape[0], "Mismatch in clean and noisy images!"

    model = unet_model(input_shape=(128, 128, 1))
    model.fit(noisy_images, clean_images, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1)
    model.save_weights(os.path.join(OUTPUT_DIR, "unet_weights.h5"))
    return model

# Visualize and save results
def visualize_and_save_results(model):
    metrics_data = []
    clean_images = load_and_resize_images(CLEAN_IMAGE_DIR, IMAGE_SIZE)
    noisy_images = load_and_resize_images(NOISY_IMAGE_DIR, IMAGE_SIZE)

    for i in range(len(clean_images)):
        original_image = clean_images[i]
        noisy_image = noisy_images[i]
        denoised_image = model.predict(np.expand_dims(noisy_image, axis=0))[0]

        psnr_value = psnr(original_image.squeeze(), denoised_image.squeeze(), data_range=1.0)
        ssim_value = ssim(original_image.squeeze(), denoised_image.squeeze(), data_range=1.0)
        mse_value = np.mean((original_image.squeeze() - denoised_image.squeeze()) ** 2)

        metrics_data.append({
            "Image Index": i + 1,
            "PSNR": psnr_value,
            "SSIM": ssim_value,
            "MSE": mse_value
        })

        # Save visualizations
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        axes[0].imshow(original_image.squeeze(), cmap='gray')
        axes[0].set_title("Original")
        axes[1].imshow(noisy_image.squeeze(), cmap='gray')
        axes[1].set_title("Noisy")
        axes[2].imshow(denoised_image.squeeze(), cmap='gray')
        axes[2].set_title("Denoised")
        plt.savefig(os.path.join(OUTPUT_DIR, f"result_{i + 1:04d}.png"))
        plt.close(fig)

    # Save metrics
    pd.DataFrame(metrics_data).to_excel(os.path.join(OUTPUT_DIR, "denoising_metrics.xlsx"), index=False)

# Main function
def main():
    create_output_directory(OUTPUT_DIR)
    model = train_model()
    visualize_and_save_results(model)

if __name__ == "__main__":
    main()


Epoch 1/2
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 222ms/step - accuracy: 0.0529 - loss: 0.0243 - val_accuracy: 0.0578 - val_loss: 0.0020
Epoch 2/2
119/119 ━━━━━━━━━━━━━━━━━━━━ 23s 196ms/step - accuracy: 0.0543 - loss: 0.0020 - val_accuracy: 0.0579 - val_loss: 0.0016
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━